In [1]:
# Step 0 : Install and import necessary libraries
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import openrouteservice
from ipyleaflet import Map, Polyline, Marker, Popup
from ipywidgets import HTML
from IPython.display import display
import time
import getpass
import folium

# Step 1: Set coordinates with labels and API key
coordinates = [
    [-8.4773019737901, 51.89801157949557, "Liberty Bar"],
    [-8.47839267379017, 51.89754580132632, "Dwyers"],
    [-8.480129616118258, 51.897412860783, "Costigans"],
    [-8.48210270262587, 51.90122750985092, "Franciscan Well"],
    [-8.478174116118428, 51.89376597099192, "Tom Barry's"],
    [-8.470903544953982, 51.901992373046355, "Corner House"],
    [-8.47113337564154, 51.90199549372547, "Sin E'"],
    [-8.4765895179699, 51.896701021615215, "An Spailpin Fanach"],
    [-8.47664922081232, 51.89677734873688, "The Oval"],
    [-8.466700360297953, 51.897178061440265, "Charlies"],
    [-8.470990660298165, 51.89379653393844, "Fionbarra"],
    [-8.469605244954147, 51.89843912054248, "The Oliver Plunkett"]
]

# Get ORS API key
api_key = getpass.getpass('Enter your OpenRouteService API key: ')
client = openrouteservice.Client(key=api_key)

# Step 2: Get distance matrix
matrix = client.distance_matrix(
    locations=[coord[:2] for coord in coordinates],  # Extract only lon, lat
    profile='foot-walking',
    metrics=['distance'],
    units='m'
)
distance_matrix = matrix['distances']

# Step 3: OR-Tools TSP solver
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

solution = routing.SolveWithParameters(search_parameters)

# Step 4: Get optimized order
optimized_coords = []
if solution:
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        optimized_coords.append(coordinates[node_index])
        index = solution.Value(routing.NextVar(index))
    optimized_coords.append(coordinates[manager.IndexToNode(index)])
else:
    print("No solution found.")
    
    # Step 5: Request actual route from ORS
route = client.directions(
    coordinates=[coord[:2] for coord in optimized_coords],  # Extract only lon, lat
    profile='foot-walking',
    format='geojson'
)

# Extract route coordinates in (lat, lon)
route_coords = [(c[1], c[0]) for c in route['features'][0]['geometry']['coordinates']]

# Step 6: Animate the route on notebook and save the final route as a static map
# Animate with ipyleaflet (notebook only)
m = Map(center=(optimized_coords[0][1], optimized_coords[0][0]), zoom=15, scroll_wheel_zoom=True, layout={'height': '700px'})
display(m)

# Add markers with labels
for lon, lat, label in optimized_coords:
    popup = Popup(location=(lat, lon), child=HTML(value=label), close_button=True, auto_close=False, close_on_click=True)
    m.add_layer(Marker(location=(lat, lon), popup=popup))

# Set animation speed
speed = 0.2  # seconds between segments

# Animate segment by segment
for i in range(len(route_coords) - 1):
    segment = [route_coords[i], route_coords[i + 1]]
    pl = Polyline(locations=segment, color="blue", weight=5)
    m.add_layer(pl)
    time.sleep(speed)

# Save static version of the map using folium
# Create folium map
fmap = folium.Map(location=[optimized_coords[0][1], optimized_coords[0][0]], zoom_start=15)

# Add markers
for lon, lat, label in optimized_coords:
    folium.Marker([lat, lon], popup=label).add_to(fmap)

# Draw the full optimized route as a single polyline
folium.PolyLine(locations=route_coords, color='blue', weight=5).add_to(fmap)

# Save to HTML
fmap.save('optimized_route_map.html')
print("Static route saved to 'optimized_route_map.html'")

# Step 7: Final report with optimized route and total distance

# Build a lookup table for fast coordinate -> index resolution
coordinate_to_index = {
    (lon, lat): idx for idx, (lon, lat, _) in enumerate(coordinates)
}

# Build route string
route_labels = [label for _, _, label in optimized_coords]
print("\nOptimized Route:")
print(" -> ".join(route_labels))

# Compute total distance
total_distance = 0
for i in range(len(optimized_coords) - 1):
    lon1, lat1, _ = optimized_coords[i]
    lon2, lat2, _ = optimized_coords[i + 1]
    idx1 = coordinate_to_index[(lon1, lat1)]
    idx2 = coordinate_to_index[(lon2, lat2)]
    total_distance += distance_matrix[idx1][idx2]

print(f"\nTotal Distance: {total_distance / 1000:.2f} km")

Map(center=[51.89801157949557, -8.4773019737901], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

Static route saved to 'optimized_route_map.html'

Optimized Route:
Liberty Bar -> Dwyers -> Costigans -> Franciscan Well -> Corner House -> Sin E' -> The Oliver Plunkett -> Charlies -> Fionbarra -> Tom Barry's -> An Spailpin Fanach -> The Oval -> Liberty Bar

Total Distance: 4.74 km
